In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

from skimage.filters.rank import entropy
from skimage.morphology import rectangle
from skimage.util import img_as_ubyte

from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

filename = 'data/facies_vectors.csv'
training_data = pd.read_csv(filename)
training_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
4144,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
4145,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
4146,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
4147,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


In [2]:
new_vars = pd.DataFrame()
l_foots = [3,5,10,20]
grouped = training_data.groupby(['Well Name'])

In [3]:
for key in grouped.groups.keys():
    
    depth = grouped.get_group(key)['Depth']
    temp_df = pd.DataFrame()
    temp_df['Depth'] = depth
    for log in ['ILD_log10','DeltaPHI','PHIND']:
      
        temp_data = grouped.get_group(key)[log]
        for l_foot in l_foots:

            footprint = rectangle(l_foot,3)

            image = np.vstack((temp_data,temp_data,temp_data))
            image -= np.median(image) 
            image /= np.max(np.abs(image))
            image = img_as_ubyte(image)
            
            if l_foot == 20:
                temp_df[log + '_entropy_foot' + str(l_foot)] = entropy(image,footprint)[0,:]
            
        #The gradient is computed using second order accurate central differences
        #in the interior and either first differences or second order accurate
        #one-sides (forward or backwards) differences at the boundaries.
        
#         temp_df[log + 'gradient1']=np.gradient(temp_data)
#         temp_df[log + 'gradient2']=np.gradient(temp_data,edge_order=2)
    
    temp_df['Well Name'] = [key for _ in range(len(depth))]
    new_vars = new_vars.append(temp_df)
                

/Users/mablou/anaconda3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [4]:
new_vars = new_vars.sort_index()

In [5]:
new_vars.to_pickle('data/vars_from_log_entropy.pkl')